In [ ]:
import torch
from transformer import vit_base_patch16_224_in21k
import torchvision.transforms as transforms
import numpy as np
from flask import Flask, jsonify, app, request
from PIL import Image
from torch import softmax
import base64
import io

app = Flask(__name__)

weights_path = "model/output/transformer_zqg_new.pt"

# 定义预处理函数，将图片进行resize和归一化等操作
preprocess = transforms.Compose(
    [  # 数据处理
        transforms.ToTensor(),  # 转换为张量
        # transforms.Resize((224, 224)),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # 归一化处理
    ]
)
# 创建模型实例
model = vit_base_patch16_224_in21k(num_classes=20, has_logits=False)
# 从.pt文件中加载权重参数
weights = torch.load(weights_path)
# 应用权重参数到模型实例
model.load_state_dict(weights)
# 获取模型实例
model = model.eval()
# 将模型转移至GPU上（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

class_names = [
    "土豆",
    "茄子",
    "香菜",
    "玉米",
    "青椒",
    "韭菜",
    "豌豆",
    "梨子",
    "苹果",
    "西兰花",
    "蒜苗",
    "水稻",
    "小麦",
    "胡萝卜",
    "葡萄",
    "南瓜",
    "黄瓜",
    "香蕉",
    "西红柿",
    "西瓜",
]

In [13]:
import cv2


img_path = "D:\\Program Files\\联想浏览器下载\\CV\\实习项目\\训练\\dataset\\train\\葡萄\\彩色手绘葡萄籽油瓶VCG211238415867.jpg"


# @app.route("/predict", methods=["POST"])
# def predict():
# print("如下是请求内容：")
# print(request.get_json())

# img_base64 = request.get_json()["imgdata"]

# image = base64.b64decode(img_base64)

# image = Image.open(io.BytesIO(image))
# image = Image.open(img_path)
data = np.uint8(np.zeros((224, 224, 3)))
data[:, :, :] = cv2.resize(
    # cv2.imread(img_path),
    cv2.imdecode(np.fromfile(img_path, dtype=np.uint8), cv2.IMREAD_COLOR),
    (224, 224),
)

image_tensor = preprocess(data).unsqueeze(0).to(device)

with torch.no_grad():

    pre = model(image_tensor)
    print(pre)

    pre = torch.softmax(pre, dim=1).cpu().numpy()[0]
    print(pre)

print(np.argsort(pre))

y_hat = np.argsort(pre)[::-1][:5]

print(y_hat)

result = []

for y in y_hat:

    name = class_names[y]
    print(name)

    score = str(pre[y] * 100) + "%"
    print(score)

    result.append([name, score])

print(result)

# return jsonify(
#     {"eror_code": "undefiened", "error_msg": "未知错误", "results": result}
# )


# 启动服务器

# if __name__ == "__main__":

#     app.run(host="0.0.0.0", port=5000)


# # 使用模型进行预测，并显示预测结果

# with torch.no_grad():

#     output = model(image_tensor)

#     _, predicted = torch.max(output, 1)

#     class_name = class_names[predicted.item()]

tensor([[ -9.4681,  -7.2270,   7.2766,   1.5104,   0.1072,   4.3938,   4.4688,
           3.9906,   9.9319,   9.4345,   3.5398, -20.7249, -10.3914,   2.7521,
          12.8537, -14.3110,  11.2272, -13.2760,  -7.5428,   7.7668]],
       device='cuda:0')
[1.5626379e-10 1.4694395e-09 2.9240947e-03 9.1575203e-06 2.2509523e-06
 1.6368876e-04 1.7643042e-04 1.0936608e-04 4.1608963e-02 2.5303096e-02
 6.9684851e-05 2.0187880e-15 6.2071785e-11 3.1697215e-05 7.7287412e-01
 1.2319692e-12 1.5195343e-01 3.4682426e-12 1.0715479e-09 4.7739726e-03]
[11 15 17 12  0 18  1  4  3 13 10  7  5  6  2 19  9  8 16 14]
[14 16  8  9 19]
葡萄
77.2874116897583%
黄瓜
15.195342898368835%
苹果
4.160896316170692%
西兰花
2.530309557914734%
西瓜
0.47739725559949875%
[['葡萄', '77.2874116897583%'], ['黄瓜', '15.195342898368835%'], ['苹果', '4.160896316170692%'], ['西兰花', '2.530309557914734%'], ['西瓜', '0.47739725559949875%']]
